In [1]:
import pandas as pd
import numpy as np
import random
import csv
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers.core import Dense , Dropout , Activation , Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers import Embedding, LSTM
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error , mean_absolute_error

In [2]:
# size of the word embeddings
embeddings_dim = 300

In [3]:
# maximum number of words to consider in the representations
max_features = 20000 #Voc is: 24535 but 10273 only (>2)

In [4]:
# maximum length of a sentence
max_sent_len = 50 # Max 58, Median = 17, avg 16

In [5]:
crossval = 20
## Random Seed
seed = 27
#number of dimensions in regression problem
reg_dimensions = 1

In [6]:
print ("")
print ("Reading pre-trained word embeddings...")
embeddings = dict( )

# Load external word embeddings
embeddings = KeyedVectors.load_word2vec_format( "twitter_sgns_subset.txt.gz" , binary=False ) 
print ("Reading text data for regression and building representations...")


Reading pre-trained word embeddings...
Reading text data for regression and building representations...


In [7]:
df = pd.read_csv("vocab.csv", sep='\t')
df

,SentID,Tweet,Emotion,Rating
0,10941,At the point today where if someone says somet...,anger,0.000
1,10942,@CorningFootball IT'S GAME DAY!!!! T MIN...,anger,0.000
2,10943,This game has pissed me off more than any othe...,anger,0.000
3,10944,@spamvicious I've just found out it's Candice ...,anger,0.000
4,10945,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,anger,0.000
...,...,...,...,...
7097,40855,Common app just randomly logged me out as I wa...,sadness,0.833
7098,40856,"I'd rather laugh with the rarest genius, in be...",sadness,0.688
7099,40857,If you #invest in my new #film I will stop ask...,sadness,0.458
7100,40858,"Just watched Django Unchained, Other people ma...",sadness,0.333


In [8]:
## We first estimate the entire vocabulary, here we use a file that combines Test&Train data:

full_voc_data  = list(df[["Tweet","Rating"]].itertuples(index=False, name=None))
full_data_size = int(len(full_voc_data))
all_texts = [txt for (txt, label) in full_voc_data[0:full_data_size] ]
tokenizer = Tokenizer(num_words=max_features, filters='%&()*+,-./:;<=>[\\]^_`{|}~\t\n',lower=True, split=" ")
tokenizer.fit_on_texts(all_texts) # <-- Tokenizer based on all TEXTS

In [9]:
test = pd.read_csv("test/anger_CNN-LSTM_input.txt", sep='\t')
test

,Tweet,Rating
0,At the point today where if someone says somet...,0.319
1,@CorningFootball IT'S GAME DAY!!!! T MIN...,0.144
2,This game has pissed me off more than any othe...,0.898
3,@spamvicious I've just found out it's Candice ...,0.271
4,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,0.646
...,...,...
755,@ggreenwald What if the supposed animosity is ...,0.646
756,Will BYU's offense score 24+ vs WVU?,0.125
757,Id love 2 c Gyimah in action but his coach is ...,0.542
758,Forgiving means operating with God's spirit &a...,0.250


In [10]:
## NOW LOAD TEST DATA ##
TSdata = list(test[["Tweet","Rating"]].itertuples(index=False, name=None))
test_size = int(len(TSdata) )
test_texts = [ txt for ( txt, label ) in TSdata[0:test_size] ]
test_labels = [ label for ( txt , label ) in TSdata[0:test_size]]
test_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( test_texts ) , maxlen=max_sent_len )
##############

In [11]:
## we Iterate through all training types, as well as the combination (afjs):
## Note that afjs is (yet) not used by our system.
EMOS="anger"
for i in EMOS.split():
    currentemo=str(i)
    temp_df = pd.read_csv("train/"+currentemo+"_tr_dv.csv", sep='\t')
    TRdata = list(temp_df[["Tweet","Rating"]].itertuples(index=False, name=None))
    random.shuffle( TRdata )    
    train_size = int(len(TRdata) )
    train_texts = [ txt for ( txt, label ) in TRdata[0:train_size] ]
    train_labels = [ label for ( txt , label ) in TRdata[0:train_size] ]
    train_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( train_texts ) , maxlen=max_sent_len )
    embedding_weights = np.zeros( ( max_features , embeddings_dim ) )
    for word,index in tokenizer.word_index.items():
        if index < max_features:
                try: embedding_weights[index,:] = embeddings[word]
                except: embedding_weights[index,:] = np.random.rand( 1 , embeddings_dim )
    np.random.seed(seed)
    filter_length = 3
    model = Sequential()
    model.add(Embedding(max_features, embeddings_dim, input_length=max_sent_len, weights=[embedding_weights]))
    model.add(Dropout(0.25))
    model.add(Convolution1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(LSTM(embeddings_dim))
    model.add(Dense(reg_dimensions))
    model.add(Activation('sigmoid'))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    train_sequences = np.array(train_sequences)
    train_labels = np.array(train_labels)
    model.fit( train_sequences , train_labels , epochs=30, batch_size=16)
    model.save("models/"+currentemo+".h5")  # creates a HDF5 file 'my_model.h5'
    

Epoch 1/30
59/59 [==============================] - 11s 102ms/step - loss: 0.1349
Epoch 2/30
59/59 [==============================] - 6s 104ms/step - loss: 0.1199
Epoch 3/30
59/59 [==============================] - 6s 109ms/step - loss: 0.0923
Epoch 4/30
59/59 [==============================] - 6s 104ms/step - loss: 0.0783
Epoch 5/30
59/59 [==============================] - 6s 109ms/step - loss: 0.0657
Epoch 6/30
59/59 [==============================] - 6s 110ms/step - loss: 0.0595
Epoch 7/30
59/59 [==============================] - 6s 104ms/step - loss: 0.0497
Epoch 8/30
59/59 [==============================] - 7s 111ms/step - loss: 0.0485
Epoch 9/30
59/59 [==============================] - 6s 109ms/step - loss: 0.0476
Epoch 10/30
59/59 [==============================] - 6s 109ms/step - loss: 0.0435
Epoch 11/30
59/59 [==============================] - 8s 127ms/step - loss: 0.0414
Epoch 12/30
59/59 [==============================] - 6s 107ms/step - loss: 0.0379
Epoch 13/30
59/59 [=====

In [12]:
test_sequences = np.array(test_sequences)
results = model.predict( test_sequences )
np.savetxt("test/pred/"+currentemo+".txt", results, newline='\n')

In [13]:
from sklearn.metrics import mean_squared_error
test_labels = np.array(test_labels)
mse = mean_squared_error(test_labels, results)

In [14]:
mse

0.018541127083854917

In [15]:
print(results)

[[0.44538257]
 [0.14478025]
 [0.61049354]
 [0.5299792 ]
 [0.5630522 ]
 [0.66149366]
 [0.4357586 ]
 [0.6455605 ]
 [0.40643853]
 [0.28805256]
 [0.5089273 ]
 [0.63296014]
 [0.4783414 ]
 [0.68764365]
 [0.5299131 ]
 [0.40814656]
 [0.41041678]
 [0.4922678 ]
 [0.54538864]
 [0.4216395 ]
 [0.6296898 ]
 [0.47792378]
 [0.48810795]
 [0.5727288 ]
 [0.4155292 ]
 [0.5438023 ]
 [0.5023271 ]
 [0.5318016 ]
 [0.59082353]
 [0.488453  ]
 [0.35725605]
 [0.5840577 ]
 [0.6805294 ]
 [0.6268372 ]
 [0.63635993]
 [0.6696046 ]
 [0.43810615]
 [0.49310887]
 [0.58263904]
 [0.7504641 ]
 [0.68578196]
 [0.5044351 ]
 [0.46258578]
 [0.6533703 ]
 [0.46990657]
 [0.71970904]
 [0.44218186]
 [0.45325464]
 [0.6122979 ]
 [0.47109008]
 [0.47262838]
 [0.660223  ]
 [0.502377  ]
 [0.3946819 ]
 [0.71369314]
 [0.3077326 ]
 [0.21013817]
 [0.5904131 ]
 [0.5087921 ]
 [0.41982448]
 [0.4799196 ]
 [0.396981  ]
 [0.39632884]
 [0.5146257 ]
 [0.49886906]
 [0.43185553]
 [0.50040865]
 [0.5062911 ]
 [0.60900927]
 [0.38495007]
 [0.40355635]
 [0.38

In [16]:
print(test_labels)

[0.319 0.144 0.898 0.271 0.646 0.583 0.375 0.625 0.396 0.25  0.438 0.708
 0.333 0.877 0.708 0.417 0.229 0.375 0.667 0.354 0.812 0.562 0.312 0.625
 0.458 0.521 0.583 0.458 0.271 0.188 0.271 0.708 0.417 0.292 0.771 0.521
 0.271 0.729 0.5   0.938 0.917 0.479 0.375 0.648 0.667 0.755 0.792 0.521
 0.479 0.5   0.417 0.5   0.396 0.333 0.798 0.375 0.173 0.771 0.625 0.906
 0.333 0.438 0.25  0.646 0.5   0.5   0.688 0.479 0.604 0.375 0.458 0.323
 0.542 0.229 0.375 0.354 0.465 0.521 0.562 0.292 0.521 0.396 0.646 0.396
 0.438 0.875 0.688 0.438 0.396 0.333 0.438 0.583 0.562 0.271 0.833 0.91
 0.771 0.851 0.296 0.292 0.479 0.458 0.521 0.492 0.312 0.292 0.562 0.583
 0.458 0.229 0.375 0.604 0.583 0.219 0.396 0.354 0.356 0.458 0.354 0.292
 0.121 0.417 0.84  0.354 0.271 0.192 0.292 0.438 0.208 0.292 0.688 0.604
 0.583 0.354 0.375 0.354 0.333 0.5   0.271 0.25  0.583 0.542 0.5   0.417
 0.688 0.417 0.542 0.688 0.312 0.521 0.396 0.375 0.333 0.729 0.708 0.688
 0.78  0.688 0.583 0.354 0.438 0.354 0.583 0.454 0.4

In [17]:
baseline = np.ones(test_labels.shape[0])*np.mean(test_labels)

In [18]:
baseline_mse = mean_squared_error(test_labels, baseline)

In [19]:
baseline_mse

0.02950585552458449